In [ ]:
import json

with open("osg_astar_mice_results.json") as f:
    data = json.load(f)

for strategy, programs in data["strategies"].items():
    print(f"\n{'=' * 70}")
    print(f"  {strategy}")
    print(f"{'=' * 70}")
    print(f"  {'#':<4} {'Latency':>10} {'Inv F1':>10} {'Not-Inv F1':>12}  Program")
    print(f"  {'-'*4} {'-'*10} {'-'*10} {'-'*12}  {'-'*30}")
    for i, p in enumerate(programs):
        print(
            f"  {i:<4} {p['latency']:>9.1f}s"
            f" {p['investigation_f1']:>10.4f}"
            f" {p['not_investigation_f1']:>12.4f}"
            f"  {p['program']}"
        )

In [ ]:
print(f"\n{'=' * 70}")
print("  Summary")
print(f"{'=' * 70}")
print(f"  {'Strategy':<30} {'First':>8} {'Last':>8} {'Best Inv':>10} {'Best NI':>10} {'Mean Inv':>10} {'Mean NI':>10}")
print(f"  {'-'*30} {'-'*8} {'-'*8} {'-'*10} {'-'*10} {'-'*10} {'-'*10}")
for strategy, programs in data["strategies"].items():
    lats = [p["latency"] for p in programs]
    inv = [p["investigation_f1"] for p in programs]
    ni = [p["not_investigation_f1"] for p in programs]
    print(
        f"  {strategy:<30}"
        f" {min(lats):>7.1f}s"
        f" {max(lats):>7.1f}s"
        f" {max(inv):>10.4f}"
        f" {max(ni):>10.4f}"
        f" {sum(inv)/len(inv):>10.4f}"
        f" {sum(ni)/len(ni):>10.4f}"
    )